<a href="https://colab.research.google.com/github/JessicaIsri/bigquery_embeddings_swivel/blob/main/BigQueryEmbenddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bigquery-ml-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 98.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 135.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.2/439.2 kB 44.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of tensorflow-text to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 108.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 118.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from bigquery_ml_utils import model_generator

text_embedding_model_generator = model_generator.TextEmbeddingModelGenerator()
text_embedding_model_generator.generate_text_embedding_model('swivel', '/content/model_output_swivel')

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [ ]:
!gsutil cp -r /content/model_output_swivel gs://bq_models_experiments/swivel

ServiceException: 401 Anonymous caller does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist).


## Carregando os modelo no Bigquery

In [ ]:
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

In [ ]:
project = 'gee-bq-experiments'
location = 'US'
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()

In [ ]:
auth.authenticate_user()

In [ ]:
%%bigquery --project gee-bq-experiments

CREATE OR REPLACE MODEL `gee-bq-experiments.ml_example.stack_overflow_swivel` OPTIONS(
  model_type='tensorflow',
  model_path='gs://bq_models_experiments/swivel/*'
)

Query is running:   0%|          |

""


## Reduzindo os dados para o ano de 2022

In [ ]:
%%bigquery --project gee-bq-experiments

create table `ml_example.posts_questions_2022` as
select
title,
regexp_replace(
    regexp_replace(
      regexp_replace(body,
        r'<img [^<>]*>',   r'(Image) '),
        r'(&)([^&;]*)(;)', r'<\2>'
      ),r'\<[^<>]*\>',    ''
    ) as text,
tags
from `gee-bq-experiments.stack_overflow.posts_questions`
WHERE EXTRACT(YEAR from creation_date) = 2022

Query is running:   0%|          |

""


In [ ]:
%%bigquery --project gee-bq-experiments

select
  title,
  text,
  tags
from `ml_example.posts_questions_2022` limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,title,text,tags
0,CCS --Passing Ports to Functions,I am starting to work on CCS and to program on...,c
1,Is this union legal?,I'm reworking and cleaning up an ex-coworker's...,c
2,Why am I receiving a floating point exception ...,Basically it's a code to input the scores and ...,c
3,"why do I get ""Segmentation fault"" when assigni...",I have this peace of C Programming code to tak...,c
4,I need to convert a char array to ULONGLONG (C...,I have an array of char and I need to convert ...,c
5,Can I put a function inside of a function?,Im currently taking a COP 3223 class and im le...,c
6,Are these lines of code in C programming the same,Are these 2 lines of code the same ??\nline 1:...,c
7,How to print the contents of an array in C?,I wanted to know how to print the contents of ...,c
8,Create Colored Text in c,I want to make a text in color in C. I tried u...,c
9,Should a C program release memory upon termina...,"In a C program, if I try to use malloc and for...",c


In [ ]:
%%bigquery --project gee-bq-experiments

SELECT * FROM
ML.PREDICT(MODEL `gee-bq-experiments.ml_example.stack_overflow_swivel`,(
SELECT "Long years ago, we made a tryst with destiny; and now the time comes when we shall redeem our pledge, not wholly or in full measure, but very substantially." AS embedding_input ))

Query is running:   0%|          |

Downloading:   0%|          |

,embedding,embedding_input
0,"[-0.09961678087711334, -1.1282159090042114, 2....","Long years ago, we made a tryst with destiny; ..."


## Definindo o calculo de distância euclidiana quadrada

In [ ]:
%%bigquery --project gee-bq-experiments

CREATE FUNCTION `ml_example.td`(a ARRAY<FLOAT64>, b ARRAY<FLOAT64>, idx INT64) AS (
   (a[OFFSET(idx)] - b[OFFSET(idx)]) * (a[OFFSET(idx)] - b[OFFSET(idx)])
);
CREATE FUNCTION `ml_example.term_distance`(a ARRAY<FLOAT64>, b ARRAY<FLOAT64>) AS ((
   SELECT SQRT(SUM( `ml_example.td`(a, b, idx))) FROM UNNEST(GENERATE_ARRAY(0, 19)) idx
));

## Busca semantica com o uso do Swivel

In [36]:
%%bigquery --project gee-bq-experiments

WITH search_term AS (
  SELECT embedding as term_embedding FROM ML.PREDICT(MODEL`gee-bq-experiments.ml_example.stack_overflow_swivel`,
  (SELECT "insert data into database" AS embedding_input))
)
SELECT

  `gee-bq-experiments.ml_example.term_distance`(term_embedding, embedding) AS termdist,
  embedding_input
FROM ML.PREDICT(MODEL `gee-bq-experiments.ml_example.stack_overflow_swivel`,(
  SELECT LOWER(title) AS embedding_input
  FROM `gee-bq-experiments.ml_example.posts_questions_2022`
)), search_term
ORDER By termdist ASC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,termdist,embedding_input
0,1.056137,bulk insert csv data using pgx.copyfrom into a...
1,1.073440,flutter: insert textfield data into the mysql ...
2,1.086497,upload csv data into sql database using asp.ne...
3,1.128166,basic inserting data into sql server database
4,1.222868,material ui media queries base on container size
5,1.228512,decode pandas columns (categorical data to met...
6,1.237413,django insert data into database via php
7,1.246065,extract data from a mysql database view
8,1.249169,add data into mysql database using eclipse
9,1.259621,angular datatables export buttons using cached...


In [70]:
%%bigquery --project gee-bq-experiments
CREATE FUNCTION `ml_example.arr_to_input_20`(arr ARRAY<FLOAT64>)
RETURNS
STRUCT<p1 FLOAT64, p2 FLOAT64, p3 FLOAT64, p4 FLOAT64,
       p5 FLOAT64, p6 FLOAT64, p7 FLOAT64, p8 FLOAT64,
       p9 FLOAT64, p10 FLOAT64, p11 FLOAT64, p12 FLOAT64,
       p13 FLOAT64, p14 FLOAT64, p15 FLOAT64, p16 FLOAT64,
       p17 FLOAT64, p18 FLOAT64, p19 FLOAT64, p20 FLOAT64>
AS (
STRUCT(
    arr[OFFSET(0)]
    , arr[OFFSET(1)]
    , arr[OFFSET(2)]
    , arr[OFFSET(3)]
    , arr[OFFSET(4)]
    , arr[OFFSET(5)]
    , arr[OFFSET(6)]
    , arr[OFFSET(7)]
    , arr[OFFSET(8)]
    , arr[OFFSET(9)]
    , arr[OFFSET(10)]
    , arr[OFFSET(11)]
    , arr[OFFSET(12)]
    , arr[OFFSET(13)]
    , arr[OFFSET(14)]
    , arr[OFFSET(15)]
    , arr[OFFSET(16)]
    , arr[OFFSET(17)]
    , arr[OFFSET(18)]
    , arr[OFFSET(19)]
));

Query is running:   0%|          |

""


In [66]:
%%bigquery --project gee-bq-experiments
CREATE  FUNCTION `ml_example.struct_to_arr_20`(s STRUCT<
    p1 FLOAT64, p2 FLOAT64, p3 FLOAT64, p4 FLOAT64,
    p5 FLOAT64, p6 FLOAT64, p7 FLOAT64, p8 FLOAT64,
    p9 FLOAT64, p10 FLOAT64, p11 FLOAT64, p12 FLOAT64,
    p13 FLOAT64, p14 FLOAT64, p15 FLOAT64, p16 FLOAT64,
    p17 FLOAT64, p18 FLOAT64, p19 FLOAT64, p20 FLOAT64>)
RETURNS ARRAY<FLOAT64>
AS (
  [s.p1, s.p2, s.p3, s.p4, s.p5, s.p6, s.p7, s.p8, s.p9, s.p10, s.p11, s.p12, s.p13, s.p14, s.p15, s.p16, s.p17, s.p18, s.p19, s.p20]
);

Query is running:   0%|          |

""


In [58]:
%%bigquery --project gee-bq-experiments
SELECT embedding, embedding_input FROM ML.PREDICT(MODEL`gee-bq-experiments.ml_example.stack_overflow_swivel`,
  (SELECT "insert data into database" AS embedding_input))

Query is running:   0%|          |

Downloading:   0%|          |

,embedding,embedding_input
0,"[1.7681946754455566, 0.2507878541946411, 0.810...",insert data into database


In [71]:
%%bigquery --project gee-bq-experiments

create or replace model `gee-bq-experiments.ml_example.text_clustering` options(
  model_type='kmeans', KMEANS_INIT_METHOD = 'KMEANS++', num_clusters=10
) as
select
  ml_example.arr_to_input_20(embedding) as text_embend,
  title
from ml.predict(MODEL `gee-bq-experiments.ml_example.stack_overflow_swivel`,(
SELECT title ,lower(title) AS embedding_input FROM `gee-bq-experiments.ml_example.posts_questions_2022` ))

Query is running:   0%|          |

""


In [72]:
%%bigquery --project gee-bq-experiments

SELECT
  centroid_id,
  text_embend
FROM
  ML.PREDICT(MODEL `gee-bq-experiments.ml_example.text_clustering`, (
  select
    ml_example.arr_to_input_20(embedding) as text_embend,
    title
  from ml.predict(MODEL`gee-bq-experiments.ml_example.stack_overflow_swivel`,
  (SELECT  "insert data into database" as title,"insert data into database" AS embedding_input))
));

Query is running:   0%|          |

Downloading:   0%|          |

,centroid_id,text_embend
0,4,"{'p1': 1.7681946754455566, 'p2': 0.25078785419..."


In [80]:
%%bigquery --project gee-bq-experiments
select * from
(SELECT
  centroid_id,
  text_embend
FROM
  ML.PREDICT(MODEL `gee-bq-experiments.ml_example.text_clustering`, (
  select
    ml_example.arr_to_input_20(embedding) as text_embend,
    title
  from ml.predict(MODEL`gee-bq-experiments.ml_example.stack_overflow_swivel`,
  (SELECT "how to make a click with python and selenium" as title,"how to make a click with python and selenium" AS embedding_input))
)));

Query is running:   0%|          |

Downloading:   0%|          |

,centroid_id,text_embend
0,2,"{'p1': 0.6437029242515564, 'p2': -0.6773621439..."


In [83]:
%%bigquery --project gee-bq-experiments

select
embedding_input search_content,
title content,
#  ml_example.struct_to_arr_10(c.text_embend),
# embedding
ML.DISTANCE(s.embedding, ml_example.struct_to_arr_20(c.text_embend), 'COSINE') AS distance
from
(SELECT embedding, embedding_input FROM ML.PREDICT(MODEL`gee-bq-experiments.ml_example.stack_overflow_swivel`,
  (SELECT "How to insert data into database" AS embedding_input))) s,
(SELECT * FROM ML.PREDICT(MODEL `gee-bq-experiments.ml_example.text_clustering`, (
  select
    ml_example.arr_to_input_20(embedding) as text_embend,
    title
  from ml.predict(MODEL `gee-bq-experiments.ml_example.stack_overflow_swivel`,(
  SELECT title ,lower(title) AS embedding_input FROM `gee-bq-experiments.ml_example.posts_questions_2022` ))
)) where centroid_id = 4) c
ORDER BY
  distance ASC
LIMIT 20;

Query is running:   0%|          |

Downloading:   0%|          |

,search_content,content,distance
0,How to insert data into database,How to add input values to database using clic...,0.028477
1,How to insert data into database,can't insert data into firebase realtime database,0.028642
2,How to insert data into database,Cant add zabbix schema and data to database,0.029870
3,How to insert data into database,How to sync data from postgres database into W...,0.032174
4,How to insert data into database,"How do I set a default scope (""Refine Scope"") ...",0.032988
5,How to insert data into database,OpenAPI Query filtering - How to filter the data,0.034264
6,How to insert data into database,Can proxy server read application layer data o...,0.036340
7,How to insert data into database,can't write data into aligned timeseries throu...,0.036855
8,How to insert data into database,MSSQL insert data info table from user defined...,0.037692
9,How to insert data into database,Template to check if vector and map contains v...,0.037778
